In [1]:
data_file_path=r'C:\Users\hp\Desktop\NLP-master\cnews.train\cnews.train.txt'
with open(data_file_path,'r',encoding='utf-8') as f:
    lines=f.readlines()

titles=[]
texts=[]
for news in lines[:10000]:
    news=news[3:]
    if('新浪体育讯' in news):
        title=news.split('新浪体育讯')[0]
        text=''.join(news.split('新浪体育讯')[1:])
        titles.append(title.strip())
        texts.append(text.strip())
print(len(titles))
print(len(texts))

4246
4246


In [2]:
print(titles[800])
print(texts[800])

布泽尔首战公牛惨负魔术 德克10分小牛大胜森林狼
北京时间12月2日，黄蜂主场力取山猫；公牛回到了久违的主场，而且迎来了卡洛斯-布泽尔，但遭到魔术的痛击；小牛首发表现平平，但仍主场轻取森林狼。以下是这三场比赛的综述：山猫73-黄蜂89黄蜂(13-5)大将克里斯-保罗只得9分，但助攻14次，大卫-韦斯特22分6个篮板，埃梅卡-奥卡福14分13个篮板。山猫(6-12)二连败。杰拉德-华莱士18分，DJ-奥古斯丁和鲍里斯-迪奥各13分。斯蒂芬-杰克逊被禁赛一场。最后3分06秒山猫又未能得分，比赛的最后9分钟他们只投中一球，仅得4分，糟糕的进攻让他们让胜利拱手相让。最后一节山猫只得11分。魔术107-公牛78魔术(14-4)五连胜。贾米尔-尼尔森拿下24分9次助攻，文斯-卡特22分，德怀特-霍华德13分12个篮板，拉沙德-刘易斯13分。替补出场的布兰顿-巴斯得了17分。公牛(9-7)回到主场，卡洛斯-布泽尔首次代表他们亮相，他首发出场，在22分钟内只得5分2个篮板。乔金-诺阿得了16分，但这个联盟第二号篮板手竟然未能抢到一个篮板，德里克-罗斯15分。夏天公牛的唯一收获就是布泽尔，但他由意外受伤，此前一直缺阵，今天才首次亮相。布泽尔表现得正像一个很久未打球的球员，未能给公牛带来太大的帮助。他们刚结束漫长的客场旅行，虽然回到主场，仍是显得疲惫。首节公牛只以22-28落后，但魔术以一波16-4开始第二节，一举以44-26拉开差距。公牛在本节打了7分钟后只投中两球，本节他们只得15分，而魔术得了33分。半场结束时，公牛以37-61落后。下半场魔术一直保持两位数的优势，最多时领先了26分。森林狼86-小牛100（点击观看视频集锦）小牛(14-4)七连胜。首发只有德克-诺维茨基和卡隆-巴特勒刚好得了10分，其他人得分都未能上双，泰森-钱德勒抢下了18个篮板。替补出场的肖恩-马里昂16分最高，杰森-特里12分，胡安-巴里亚14分。森林狼(4-14)五连败。迈克尔-比斯利16分，凯文-勒夫12分15个篮板，达科-米利西奇12分8个篮板。小牛首发五虎状态一般，诺维茨基也一反此前连砍高分的常态，开场后命中率不高。尽管如此，小牛还是早早控制局面，在第二节还有2人22秒时以53-37领先。半场结束时，森林狼只能将比分追成43-55。下半场小牛继续扩大优势，第三节结束前卡迪纳尔三分命中，他们以83-

In [3]:
titles_lens=[]
texts_lens=[]
for line in titles:
    titles_lens.append(len(line))
for line in texts:
    texts_lens.append(len(line))
titles_lens.sort()
print('title_len_avg:%f'%(sum(titles_lens)/len(titles)))
print('title_len_middle:%f'%(titles_lens[int(len(titles)/2)]))
print('title_len_min:%f'%(titles_lens[0]))
print('title_len_max:%f'%(titles_lens[len(titles)-1]))
texts_lens.sort()
print('text_len_avg:%f'%(sum(texts_lens)/len(texts)))
print('text_len_middle:%f'%(texts_lens[int(len(texts)/2)]))
print('text_len_min:%f'%(texts_lens[0]))
print('text_len_max:%f'%(texts_lens[len(texts)-1]))


title_len_avg:23.757654
title_len_middle:23.000000
title_len_min:13.000000
title_len_max:30.000000
text_len_avg:838.758832
text_len_middle:813.000000
text_len_min:48.000000
text_len_max:12395.000000


In [4]:
import jieba
from keras.models import Model
from keras.layers import Input, LSTM, Dense,Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
MAX_WORDS=10000

#将句子分词并用空格隔开
inputTextList=[' '.join([w for w in jieba.cut(text)]) for text in texts]
targetTextList=[' '.join([w for w in jieba.cut(text)]) for text in titles]

#-4的原因是后面会加上4个特殊字符串，值从1开始，故词典里只设置9999个词
tokenizer=Tokenizer(num_words=MAX_WORDS-5)
tokenizer.fit_on_texts(texts=inputTextList+targetTextList)

word_index=tokenizer.word_index

# 增加特殊编码
SPECIAL_CODES = ['<PAD>', '<EOS>', '<UNK>', '<GO>']

for i,w in enumerate(SPECIAL_CODES):
    word_index[w]=MAX_WORDS-i-1

re_word_index = dict([(i, t) for t, i in word_index.items()])

with open('AutoDigestGeneration_Dict.txt','w',encoding='utf-8') as f:
    f.write(str(word_index))

#将文本映射为数字序列
input_sequences=tokenizer.texts_to_sequences(texts=inputTextList)
target_sequences=tokenizer.texts_to_sequences(texts=targetTextList)

print(len(word_index))
print(len(re_word_index))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Public\Documents\Wondershare\CreatorTemp\jieba.cache
Loading model cost 0.537 seconds.
Prefix dict has been built successfully.


46706
46702


In [6]:
print(input_sequences[800])

for n in input_sequences[800]:
    print(re_word_index[n],end='')

[82, 28, 79, 37, 40, 49, 1, 297, 103, 2621, 366, 206, 197, 397, 5, 4068, 2, 103, 1, 266, 769, 5, 1328, 1115, 1, 27, 998, 175, 2, 6028, 206, 213, 176, 65, 2659, 1, 27, 419, 103, 2154, 385, 3, 620, 7, 23, 781, 9, 2, 1708, 16, 366, 1864, 297, 985, 297, 141, 56, 1421, 439, 291, 867, 87, 8, 1, 27, 59, 156, 36, 1, 883, 803, 273, 8, 63, 30, 26, 1, 3433, 1898, 156, 8, 141, 30, 26, 3, 366, 63, 79, 614, 212, 3, 1925, 881, 207, 8, 1, 3598, 2593, 11, 4155, 2709, 1501, 141, 8, 3, 1844, 480, 48, 2283, 146, 3, 86, 39, 8, 1900, 75, 366, 94, 517, 85, 1, 9, 2, 86, 87, 125, 17, 166, 307, 251, 172, 1, 6217, 53, 8, 1, 760, 2, 66, 41, 17, 41, 229, 3, 86, 623, 366, 867, 81, 8, 3, 175, 1601, 197, 1672, 175, 156, 53, 938, 214, 3, 608, 812, 1151, 122, 268, 8, 87, 36, 59, 1, 2603, 937, 273, 8, 1, 1064, 260, 141, 8, 79, 30, 26, 1, 1447, 376, 141, 8, 3, 173, 170, 2, 217, 1406, 2108, 91, 5, 201, 8, 3, 197, 87, 68, 397, 103, 1, 1328, 1115, 585, 1036, 17, 2069, 1, 6, 176, 170, 1, 4, 273, 125, 429, 867, 56, 8, 40, 30,

In [7]:
titles_lens=[]
texts_lens=[]
for line in input_sequences:
    titles_lens.append(len(line))
for line in target_sequences:
    texts_lens.append(len(line))
titles_lens.sort()
print('title_len_avg:%f'%(sum(titles_lens)/len(titles)))
print('title_len_middle:%f'%(titles_lens[int(len(titles)/2)]))
print('title_len_min:%f'%(titles_lens[0]))
print('title_len_max:%f'%(titles_lens[len(titles)-1]))
texts_lens.sort()
print('text_len_avg:%f'%(sum(texts_lens)/len(texts)))
print('text_len_middle:%f'%(texts_lens[int(len(texts)/2)]))
print('text_len_min:%f'%(texts_lens[0]))
print('text_len_max:%f'%(texts_lens[len(texts)-1]))

title_len_avg:478.819830
title_len_middle:465.000000
title_len_min:26.000000
title_len_max:6457.000000
text_len_avg:10.961611
text_len_middle:11.000000
text_len_min:3.000000
text_len_max:19.000000


In [8]:
#因此确定
Tx=500
Ty=20

In [9]:
#padding数据
import tqdm

input_arr=[]
target_arr=[]

for line in tqdm.tqdm(input_sequences):
    slen=len(line)
    if(slen<Tx):
        newline=line+[word_index['<PAD>']]*(Tx-slen)
        input_arr.append(newline)
    else:
        input_arr.append(line[:Tx])


for line in tqdm.tqdm(target_sequences):
    slen=len(line)
    if(slen<Ty):
        line.append(word_index['<EOS>'])
        newline=line+[word_index['<PAD>']]*(Ty-slen-1)
        target_arr.append(newline)
    else:
        target_arr.append(line[:Ty])

input_arr=np.array(input_arr)
target_arr=np.array(target_arr)

print(input_arr.shape)
print(target_arr.shape)

100%|██████████████████████████████████████████████████████████████████████████| 4246/4246 [00:00<00:00, 607360.17it/s]

(4246, 500)
(4246, 20)


In [10]:
print(input_arr[800])
print(target_arr[800])

[ 372 2461  735 1206   38 2543  398 1555    1   27    6  851  290    2
 2085   51 4918    5    3  317  387  443    2  133    1  735  106   51
  670    5   11  371    2 4305    1  496   33    4 3937  516  397 1106
    3 4318    1  710   94   67  348 1198  642 4479 7282  816    2 3703
  104    6    4  588 3166 7362    5    3  735   11 8338  135    6    2
 9756   20    7  132   93  107    2 1797    1 8338  251   20   61  178
  118  735   11    6    2   19 8777    1 1137   10  733  165    5  735
    2 3695    3  501    1 1137   10    7  128   14  505  680    2 3085
 2372  921    1  441    2    3 8338 7741  538   64   24  519   43  556
    2  857 1089    1 1137  141  531  112    1 1825  102  652 7253  374
  578 1965  383   21  374  383  426 6680    3  145 4119   60 5548    2
   12   13 3851  302    1   27 1137    2 3693 2672    7   14  152    2
 9425    3  167 1721 1349    5  374  383 7765 4254    2    1    7  609
   12  152 4479    2   13  535    3   18  154  335  164 1137 7438    7
    2 

In [11]:
from keras import backend as K
# 自定义softmax函数
def softmax(x, axis=1):
    ndim = K.ndim(x)
    if ndim == 2:
        return K.softmax(x)
    elif ndim > 2:
        e = K.exp(x - K.max(x, axis=axis, keepdims=True))
        s = K.sum(e, axis=axis, keepdims=True)
        return e / s
    else:
        raise ValueError('Cannot apply softmax to a tensor that is 1D')
        

In [12]:
from keras.layers import *

# 定义全局网络层对象
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor_tanh = Dense(32, activation = "tanh")
densor_relu = Dense(1, activation = "relu")
densor_con = Dense(256, activation = "relu")
activator = Activation(softmax, name='attention_weights')
dotor = Dot(axes = 1)

In [13]:
def one_step_attention(a, s_prev):
    # 将s_prev复制Tx次
    s_prev = repeator(s_prev)
    # 拼接BiRNN隐层状态与s_prev
    concat = concatenator([a, s_prev])
    # 计算energies
    e = densor_tanh(concat)
    energies = densor_relu(e)
    # 计算weights
    alphas = activator(energies)
    # 加权得到Context Vector
    context = dotor([alphas, a])
    
    return context

In [14]:
# 加载预训练好的glove词向量
with open(r'C:\Users\hp\Desktop\NLP-master\sgns.zhihu.word\sgns.zhihu.word', 'r',encoding='utf-8') as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
        line = line.strip().split()
        curr_word = line[0]
        words.add(curr_word)
        word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)

In [15]:
def pretrained_embedding_layer(word_to_vec_map, word_index):
    vocab_len = len(word_index) + 1        # Keras Embedding的API要求+1
    emb_dim = 300
    # 初始化embedding矩阵
    emb_matrix = np.zeros((vocab_len, emb_dim))
    # 用词向量填充embedding矩阵
    for word, index in word_index.items():
        word_vector = word_to_vec_map.get(word, np.zeros(emb_dim))
        emb_matrix[index, :] = word_vector
    # 定义Embedding层，并指定不需要训练该层的权重
    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)
    # build
    embedding_layer.build((None,))
    # set weights
    embedding_layer.set_weights([emb_matrix])
    return embedding_layer

In [16]:
# 获取Embedding layer
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_index)

In [17]:
from tensorflow.keras.utils import to_categorical

def train_gen(X,Y,Ty,n_s,batch_size=64):
    xlen=X.shape[0]
    permutation = np.random.permutation(xlen)
    x = X[permutation]
    y = Y[permutation]
    num_batches = int(xlen/batch_size)
    
    while 1:
        for i in range(num_batches):
            x_batch=x[i*batch_size:(i+1)*batch_size]
            y_batch=y[i*batch_size:(i+1)*batch_size]
            s0=np.zeros((batch_size,n_s))
            c0=np.zeros((batch_size,n_s))
            out0=np.zeros((batch_size,MAX_WORDS))#加一的原因是字典中的值从1开始
            outputs=np.zeros((batch_size,Ty,MAX_WORDS))
            for i,line in enumerate(y_batch):
                for j,index in enumerate(line):
                    outputs[i,j,index]=1
                    
            #outputs=np.array(list(map(lambda x: to_categorical(x, num_classes=MAX_WORDS), y_batch)))
            outputs = list(outputs.swapaxes(0,1))
            yield [x_batch,s0,c0,out0],outputs

In [18]:
n_a = 128 # The hidden size of Bi-LSTM
n_s = 128 # The hidden size of LSTM in Decoder
decoder_LSTM_cell = LSTM(n_s, return_state=True)
output_layer = Dense(MAX_WORDS, activation=softmax)

In [19]:
# 定义网络层对象（用在model函数中）
reshapor = Reshape((1, MAX_WORDS))
concator = Concatenate(axis=-1)

In [20]:
def model(Tx, Ty, n_encoder, n_decoder):
    """
    构造模型
    @param Tx: 输入序列的长度
    @param Ty: 输出序列的长度
    @param n_encoder: Encoder端Bi-LSTM隐层结点数
    @param n_decoder: Decoder端LSTM隐层结点数
    """
    
    # 定义输入层
    X = Input(shape=(Tx,))
    # Embedding层
    embed = embedding_layer(X)
    
    # 定义Bi-LSTM
    a = Bidirectional(LSTM(n_decoder, return_sequences=True))(embed)
    
    # Decoder端LSTM的初始状态
    s0 = Input(shape=(n_decoder,), name='s0')
    c0 = Input(shape=(n_decoder,), name='c0')
    
    # Decoder端LSTM的初始输入
    out0 = Input(shape=(MAX_WORDS, ), name='out0')
    out = reshapor(out0)
    
    s = s0
    c = c0
    
    # 模型输出列表，用来存储翻译的结果
    outputs = []
    
    # Decoder端，迭代Ty轮，每轮生成一个翻译结果
    for t in range(Ty):
        # 获取Context Vector
        context = one_step_attention(a, s)
        
        # 将Context Vector与上一轮的翻译结果进行concat
        con_out=densor_con(reshapor(out))
        context = concator([context,con_out])
        s, _, c = decoder_LSTM_cell(context, initial_state=[s, c])
        
        # 将LSTM的输出结果与全连接层链接
        out = output_layer(s)
        
        # 存储输出结果
        outputs.append(out)
    
    model = Model([X, s0, c0, out0], outputs)
    
    return model

In [21]:
model = model(Tx, Ty, n_a, n_s)

In [22]:
from keras.utils.vis_utils import plot_model
plot_model(model,to_file='attentionTest_model.png',show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [23]:
#model.summary()

In [24]:
from keras.optimizers import adam_v2
from keras.models import load_model
import keras.backend as K
'''
out = model.compile(optimizer=Adam(lr=0.1, beta_1=0.9, beta_2=0.999, decay=0.001),
                    metrics=['accuracy'],
                    loss='categorical_crossentropy')
'''
out = model.compile(optimizer='rmsprop',
                    metrics=['accuracy'],
                    loss='categorical_crossentropy')

In [25]:
# 训练模型
xlen=len(input_arr)

model.fit_generator(train_gen(input_arr,target_arr,Ty,n_s,batch_size=8),
                   steps_per_epoch=int(xlen/4),
                   epochs=3)

Epoch 1/3


C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_11556/3907132207.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_gen(input_arr,target_arr,Ty,n_s,batch_size=8),


1061/1061 [==============================] - 956s 872ms/step - loss: 88.8389 - dense_3_loss: 6.6072 - dense_3_1_loss: 7.0508 - dense_3_2_loss: 7.3391 - dense_3_3_loss: 7.5372 - dense_3_4_loss: 7.5707 - dense_3_5_loss: 7.5989 - dense_3_6_loss: 7.5369 - dense_3_7_loss: 7.4504 - dense_3_8_loss: 7.1082 - dense_3_9_loss: 6.6295 - dense_3_10_loss: 5.5975 - dense_3_11_loss: 4.2424 - dense_3_12_loss: 2.8150 - dense_3_13_loss: 1.6236 - dense_3_14_loss: 0.8551 - dense_3_15_loss: 0.4569 - dense_3_16_loss: 0.2783 - dense_3_17_loss: 0.2012 - dense_3_18_loss: 0.1742 - dense_3_19_loss: 0.1658 - dense_3_accuracy: 0.0411 - dense_3_1_accuracy: 0.1115 - dense_3_2_accuracy: 0.0375 - dense_3_3_accuracy: 0.0254 - dense_3_4_accuracy: 0.0398 - dense_3_5_accuracy: 0.0221 - dense_3_6_accuracy: 0.0180 - dense_3_7_accuracy: 0.0310 - dense_3_8_accuracy: 0.0607 - dense_3_9_accuracy: 0.1165 - dense_3_10_accuracy: 0.2258 - dense_3_11_accuracy: 0.3901 - dense_3_12_accuracy: 0.5871 - dense_3_13_accuracy: 0.7687 - dense

In [30]:
model.save('AutoDigestGenerationTest.h5')
model.save_weights('AutoDigestGenerationTest_weights.h5')

In [31]:
from keras.models import load_model

model=load_model('AutoDigestGenerationTest.h5')

In [32]:
import jieba

with open('AutoDigestGeneration_Dict.txt','r',encoding='utf-8') as f:
    word_index_str=f.read()
    

    
word_index=eval(word_index_str)
re_word_index = dict([(i, t) for t, i in word_index.items()])
Tx=500
n_decoder=128

def make_prediction(sentence):
    # 将句子分词后转化为数字编码
    input_seq=[]
    for w in jieba.cut(sentence):
        if w in word_index:
            input_seq.append(word_index[w])
            
    if(len(input_seq)<=Tx):
        input_seq = np.array(input_seq + [word_index['PAD']] * (Tx - len(input_seq)))
    else:
        input_seq = np.array(input_seq[:Tx])
    s0=np.zeros((1,n_decoder))
    c0=np.zeros((1,n_decoder))
    out0=np.zeros((1,MAX_WORDS))
    
    print(input_seq)
    
    # 翻译结果
    preds = model.predict([input_seq.reshape(-1,Tx), s0, c0, out0])
    predictions = np.argmax(preds, axis=-1)
    
    print(predictions)
    
    # 转换为单词
    idx = [re_word_index.get(idx[0], "<UNK>") for idx in predictions]
    
    # 返回句子
    return " ".join(idx)


text='''
北京时间12月2日，黄蜂主场力取山猫；公牛回到了久违的主场，而且迎来了卡洛斯布泽尔，但遭到魔术的痛击；小牛首发表现平平，但仍主场轻取森林狼。以下是这三场比赛的综述：山猫73黄蜂89黄蜂135大将克里斯保罗只得9分，但助攻14次，大卫韦斯特22分6个篮板，埃梅卡奥卡福14分13个篮板。山猫612二连败。杰拉德华莱士18分，dj奥古斯丁和鲍里斯迪奥各13分。斯蒂芬杰克逊被禁赛一场。最后3分06秒山猫又未能得分，比赛的最后9分钟他们只投中一球，仅得4分，糟糕的进攻让他们让胜利。最后一节山猫只得11分。魔术107公牛78魔术144五连胜。贾米尔尼尔森拿下24分9次助攻，文斯卡特22分，德怀特霍华德13分12个篮板，拉沙德刘易斯13分。替补出场的布兰顿巴斯得了17分。公牛97回到主场，卡洛斯布泽尔首次代表他们亮相，他首发出场，在22分钟内只得5分2个篮板。乔金了16分，但这个联盟篮板手竟然未能抢到一个篮板，德里克罗斯15分。夏天公牛的唯一收获就是布泽尔，但他由意外受伤，此前一直缺阵，今天才首次亮相。布泽尔表现得一个未打球的球员，未能给公牛带来太大的帮助。他们刚结束漫长的客场旅行，虽然回到主场，仍是显得疲惫。首节公牛只以2228落后，但魔术以一波164开始第二节，一举以4426拉开差距。公牛在本节打了7分钟后只投中两球，本节他们只得15分，而魔术得了33分。半场结束时，公牛以3761落后。下半场魔术一直保持两位数的优势，最多时领先了26分。森林狼86小牛100（点击观看视频集锦）小牛144七连胜。首发只有德克诺维茨基和卡隆巴特勒刚好得了10分，其他人得分都未能上双，泰森钱德勒抢下了18个篮板。替补出场的肖恩马里昂16分最高，杰森特里12分，胡安巴里亚14分。森林狼414五连败。迈克尔比斯利16分，凯文勒夫12分15个篮板，达科米利西奇12分8个篮板。小牛首发五虎状态一般，诺维茨基也此前连砍高分的，开场后命中率不高。尽管如此，小牛还是早早控制局面，在第二节还有2人22秒时以5337领先。半场结束时，森林狼只能将比分追成4355。下半场小牛继续扩大优势，第三节结束三分命中，他们以8364领先了19分。大比分领先的小牛第四节给了替补更多机会，他们很快取得20分以上的优势，早早锁定胜局。吴哥'''



print(text)
result=make_prediction(text)
print(result)


北京时间12月2日，黄蜂主场力取山猫；公牛回到了久违的主场，而且迎来了卡洛斯布泽尔，但遭到魔术的痛击；小牛首发表现平平，但仍主场轻取森林狼。以下是这三场比赛的综述：山猫73黄蜂89黄蜂135大将克里斯保罗只得9分，但助攻14次，大卫韦斯特22分6个篮板，埃梅卡奥卡福14分13个篮板。山猫612二连败。杰拉德华莱士18分，dj奥古斯丁和鲍里斯迪奥各13分。斯蒂芬杰克逊被禁赛一场。最后3分06秒山猫又未能得分，比赛的最后9分钟他们只投中一球，仅得4分，糟糕的进攻让他们让胜利。最后一节山猫只得11分。魔术107公牛78魔术144五连胜。贾米尔尼尔森拿下24分9次助攻，文斯卡特22分，德怀特霍华德13分12个篮板，拉沙德刘易斯13分。替补出场的布兰顿巴斯得了17分。公牛97回到主场，卡洛斯布泽尔首次代表他们亮相，他首发出场，在22分钟内只得5分2个篮板。乔金了16分，但这个联盟篮板手竟然未能抢到一个篮板，德里克罗斯15分。夏天公牛的唯一收获就是布泽尔，但他由意外受伤，此前一直缺阵，今天才首次亮相。布泽尔表现得一个未打球的球员，未能给公牛带来太大的帮助。他们刚结束漫长的客场旅行，虽然回到主场，仍是显得疲惫。首节公牛只以2228落后，但魔术以一波164开始第二节，一举以4426拉开差距。公牛在本节打了7分钟后只投中两球，本节他们只得15分，而魔术得了33分。半场结束时，公牛以3761落后。下半场魔术一直保持两位数的优势，最多时领先了26分。森林狼86小牛100（点击观看视频集锦）小牛144七连胜。首发只有德克诺维茨基和卡隆巴特勒刚好得了10分，其他人得分都未能上双，泰森钱德勒抢下了18个篮板。替补出场的肖恩马里昂16分最高，杰森特里12分，胡安巴里亚14分。森林狼414五连败。迈克尔比斯利16分，凯文勒夫12分15个篮板，达科米利西奇12分8个篮板。小牛首发五虎状态一般，诺维茨基也此前连砍高分的，开场后命中率不高。尽管如此，小牛还是早早控制局面，在第二节还有2人22秒时以5337领先。半场结束时，森林狼只能将比分追成4355。下半场小牛继续扩大优势，第三节结束三分命中，他们以8364领先了19分。大比分领先的小牛第四节给了替补更多机会，他们很快取得20分以上的优势，早早锁定胜局。吴哥
[   82    28    79    37    40    49     1   2

In [33]:
text2='''
公牛遭到魔术痛击，小牛胜森林狼
'''
print(text2)


公牛遭到魔术痛击，小牛胜森林狼

